In [1]:
# imports
import numpy as np
import seaborn as sns
import tqdm
import random
import matplotlib.pyplot as plt
import os
import pandas as pd
from numpy.random import choice
from scipy import stats
from collections import Counter
from model import AbstractCell, Action, Brain, Submodels
from modelrgpn import BrainLI
from experiment import Experiment
from utils import nop, highest_lower, Profiler, plot_function
from submodels.cellbasic1 import CellBasic
from submodels.bistate1 import BiStateModelFactory
from submodels.bistate_LI import BiStateLIModelFactory
from submodels.tristate_LI import TriStateLIModelFactory
from submodels import factories
from biodata import *
from tree import tree_from_cell

In [2]:
def var_metric(gpn, cells):
    ls_ratio = list()
    for cell in cells:
        ngbs = gpn.ngb(cell.gpn_id)
        # now dependent of the data format
        ratio = np.mean([ngb == IP])

## Model

In [3]:
# Definition of the var of the model
# We start with 100 cells for homogeneity
START_POPULATION_SQRT = 10
START_TIME = 49
END_TIME = 94
# We arbitrarily set variance of Tc at 12.5^2
# 95 % seems to be into 50, so sigma = 50 / 4 = 12.5
SIG_TC = 12.5

In [4]:
# attempt to show side collapse

In [5]:
smooth = 0.2
start_val = 0.35
bias_ratio = [0.2, 0.2, -0.1, -0.1, -0.1]
size = 6
NAME = "none"
DESC = f"bias {bias_ratio} size {size} startval {start_val} smooth {smooth}"

In [6]:
root = "/home/nathan/Bureau/ENS/Thèse/dev/EmbryonicCortexModelling/output/evonet13/"
count = 0
from viz import embedded_viz, quick_export
import gc
def my_callback(gpn):
    global count, root
    gc.collect()
    count += 1
    if count % 100 == 0:
        print(count)
        
    name = os.path.join(root, f"image_{count}.png")
    
    if os.path.exists(name):
        return
    # here
    gpn.update_dist()
    pos = embedded_viz(gpn, finetune=False)
    quick_export(gpn, pos, name)

In [7]:
def go(seed=0):
    random.seed(0+seed)
    np.random.seed(0+seed)
    ccls = TriStateLIModelFactory(tc_coeff_RG=[1., 1., 1., 1., 1.], smooth=smooth, start_val=start_val,
                                bias_ratio=bias_ratio)
    bb = BrainLI(time_step=0.5, verbose=True, start_population=size, cell_cls=ccls.generate, check=True,
                end_time=94, callback_gpn=None)
    # bb.gpn.debug = True

    # bb.run()
    return bb

In [8]:
bb = go(0)
bb.run()

Ticking abs : 49.0, step : 0.5
Ticking abs : 49.5, step : 0.5
Duplicating 12 12
Duplicated with ids : 12 and 36
Duplicating 16 16
Duplicated with ids : 16 and 37
Duplicating 17 17
Duplicated with ids : 17 and 38
Duplicating 18 18
Duplicated with ids : 18 and 39
Duplicating 19 19
Duplicated with ids : 19 and 40
Duplicating 20 20
Duplicated with ids : 20 and 41
Duplicating 21 21
Duplicated with ids : 21 and 42
Duplicating 23 23
Duplicated with ids : 23 and 43
Duplicating 24 24
Duplicated with ids : 24 and 44
Duplicating 27 27
Counter at 10 1628152611.562607
Duplicated with ids : 27 and 45
Duplicating 30 30
Duplicated with ids : 30 and 46
Duplicating 31 31
Duplicated with ids : 31 and 47
Ticking abs : 50.0, step : 0.5
Duplicating 8 8
Duplicated with ids : 8 and 48
Duplicating 9 9
Duplicated with ids : 9 and 49
Duplicating 14 14
Duplicated with ids : 14 and 50
Duplicating 15 15
Duplicated with ids : 15 and 51
Removing 39 16
Removing 41 17
Removing 49 21
Removing 51 23
Counter at 20 1628152

Removing 251 36
Duplicating 189 44
Duplicated with ids : 44 and 157
Duplicating 151 54
Counter at 200 1628152629.2333646
Duplicated with ids : 54 and 158
Duplicating 177 65
Duplicated with ids : 65 and 159
Duplicating 195 71
Duplicated with ids : 71 and 160
Duplicating 241 82
Duplicated with ids : 82 and 161


KeyboardInterrupt: 

In [ ]:
raise

In [ ]:
bb.gpn.check_not_double_border()

In [ ]:
for x in [1846] + list(bb.gpn.ngb(1846)):
    print(x)
    bb.gpn.summary(x)

In [ ]:
bb.gpn.summary(1831)

In [ ]:
# faulty node is 1846, now 615

In [ ]:
bb.gpn.summary(1039)

In [ ]:
bb.gpn.summary(1914)

In [ ]:
for n in bb.gpn.G:
    if len(bb.gpn.G.nodes[n]["ngb"]) <= 2:
        print(n, bb.gpn.is_border_node(n))

In [ ]:
for n in bb.gpn.ngb(1914):
    print(n, bb.gpn.is_border_node(n))

In [ ]:
bb.gpn.get_cycle_pairs(1914)

In [ ]:
bb.gpn.dual((7302, -1, 1)), bb.gpn.dual((7302, -1, 0))

In [ ]:
bb.gpn.ngb(1844), bb.gpn.ngb(1896)

In [ ]:
bb.gpn.is_border_node(1708)

In [ ]:
bb.gpn.show_all()

In [ ]:
len(bb.gpn.G)

In [ ]:
raise

In [ ]:
bb.gpn.update_all_dist()

In [ ]:
for n in bb.gpn.G.nodes:
    node = bb.gpn.G.nodes[n]
    if node["dist_E"] + node["dist_W"] == 0:
        print(n)

In [ ]:
mg = bb.gpn.copy()

In [ ]:
sel = 629

In [ ]:
mg.G.nodes[341]

In [ ]:
mg.ngb(341)

In [ ]:
mg.G.nodes[341]["side"] = {'W', 'S'}

In [ ]:
import viz
mg.update_all_dist()
pos = viz.embedded_viz(mg, finetune=False)
fig = mg.show_dist(pos=pos, hl_nodes=[1914, 1896, 1844, 1708])

In [ ]:
from relaxablegpn2 import EnergyPoint
def relax(self):
    # prepare
    self.last_action = None
    self.quick_update_all_dist()
    mean_NS = self.mean_NS()
    mean_EW = self.mean_EW()

    nodes = list(self.G.nodes)
    random.shuffle(nodes)

    # list all pain points
    candidates = list()
    for n in nodes:
        node = self.G.nodes[n]
        dist_NS = node["dist_N"] + node["dist_S"]
        dist_EW = node["dist_E"] + node["dist_W"]
        val_NS = dist_NS / mean_NS
        val_EW = dist_EW / mean_EW
        if dist_NS < 4:
            candidates.append(EnergyPoint(n, "NS", self.low_width_penalty(dist_NS)))
        elif val_NS >= 1.1 or val_NS <= 0.9:
            candidates.append(EnergyPoint(n, "NS", val_NS))

        if dist_EW < 4:
            candidates.append(EnergyPoint(n, "EW", self.low_width_penalty(dist_EW)))
        elif val_EW >= 1.1 or val_EW <= 0.9:
            candidates.append(EnergyPoint(n, "EW", val_EW))

    # lower is worse, so to be tested

    candidates.sort(reverse=False)

    # list all possible actions
    action_candidates = list()
    for pt in candidates[:5]:
        for action in self.enumerate_actions(pt):
            self.estimate_action(action, mean_NS, mean_EW)
            action_candidates.append(action)

    # higher score is better to do !!

    action_candidates.sort(reverse=True)
    return candidates, action_candidates

In [ ]:
cand, act = relax(mg)

In [ ]:
mg.low_width_penalty(2)

In [ ]:
cand[:10]

In [ ]:
mg.G.nodes[805]

In [ ]:
cand[0].to_shorten()

In [ ]:
list(mg.enumerate_actions(cand[1]))

In [ ]:
act

In [ ]:
mg.ngb(2382), mg.ngb(2506)

In [ ]:
def act_from_one_c(self, pt):
    action_candidates = []
    for action in self.enumerate_actions(pt):
        self.estimate_action(action, 5, 5)
        action_candidates.append(action)
    action_candidates.sort(reverse=True)
    return action_candidates

In [ ]:
act_from_one_c(mg, cand[0])

In [ ]:
raise

In [ ]:
name = os.path.join(root, f"image_{count}.png")seed = 0
while True:
    print("Doing", seed)
    try:
        bb = go(seed)
    except:
        seed += 1
        if seed > 15:
            raise
    else:
        break

In [ ]:
def plot_size(stats, name=None):
    ref = stats.whole_pop_size.iloc[0]
    plot_number_cells()
    p1 = plt.plot(stats.time, stats.progenitor_pop_size / ref, label="Progenitor population")
    # p2 = plt.plot(stats.time, stats.whole_pop_size / ref, label="Whole Population")
    plt.legend()
    if name:
        plt.savefig(f"output/results/prog_size_{name}.png")
    
def plot_ratio(stats, name=None):
    non_IP = (stats.size_type_RG + stats.size_type_GP) if "size_type_GP" in stats.columns else \
                stats.size_type_RG
    
    plt.plot(stats.time, stats.size_type_IP / (non_IP + stats.size_type_IP),
             label="IP ratio")
    plt.plot(ratio_eomes.index, ratio_eomes.val / 100, label="Reference IP ratio")
    plt.legend()
    if name:
        plt.savefig(f"output/results/IP_ratio_{name}.png")
        
def plot_size_ratio(stats, name=None):
    plt.figure(figsize=(12, 12))
    plt.subplot(2, 1, 1)
    plot_size(stats)
    plt.subplot(2, 1, 2)
    plot_ratio(stats)
    if name:
        plt.savefig(f"output/results/prog_size_ratio_{name}.png")
        
def save_stats(stats, name):
    stats.to_csv(f"output/results/stats_{name}.csv")
    
def save_txt(txt, name):
    fn = f"output/results/txt_{name}.txt"
    with open(fn, "w") as f:
        f.write(txt)

In [ ]:
plot_size_ratio(bb.stats, NAME)

In [ ]:
save_stats(bb.stats, NAME)
save_txt(DESC, NAME)